In [1]:
# Заранее импортируем модули, которые понадобятся нам для решения задачи:
import numpy as np
import pandas as pd
import plotly.express as px
import re
import category_encoders as ce

In [2]:
# Прочитаем файл с исходными данными:
data = pd.read_csv("data/data.csv")

In [3]:
# Проанализируем датасет на наличие дублирующихся записей
# data = df_train
dupl_columns = list(data.columns)
# dupl_columns.remove('id')

mask = data.duplicated(subset=dupl_columns)
data_duplicates = data[mask]
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')


Число найденных дубликатов: 50


In [4]:
# удалим дубликаты
data = data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data.shape[0]}')

Результирующее число записей: 377135


In [5]:
# Проверим датасет на наличие пропусков в данных
cols_null_percent = data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent > 0].sort_values(
    ascending=False)
display(cols_with_null)
# data.isnull().sum()

private pool    98.891378
mls-id          93.386453
PrivatePool     89.311520
fireplace       72.659127
stories         39.952007
baths           28.188315
beds            24.196640
MlsId           17.730786
sqft            10.752118
status          10.584274
propertyType     9.209699
target           0.657589
city             0.009015
street           0.000530
dtype: float64

In [6]:
# удалим строки, в которых отсутствуют значения целевого признака

data.dropna(subset=['target'], inplace=True)

In [7]:
# функция разбиения значений признака: 

def set_sign(col, key):
    delimiters = " |/|, |: |\n"
    # data_list = col.split(' ')
    data_list = re.split(delimiters, col)
    if key in data_list:
        return key
    else: return col

In [8]:
# функция объединения близких по смыслу значений признака по ключевым словам:

def comb_val(col, lst):
    for item in lst:
        data[col] = data[col].astype('str').apply(set_sign, key=item)

In [9]:
# функция переназначения значений признака по словарю:

def remap_val(dict, col):
        data[col] = data[col].replace(dict)

In [10]:
# функция вывода количества уникальных значений признака:

def uniq_val_count(col):
    # pd.set_option("display.max_rows", None)
    # col = 'status'
    print(data[col].value_counts(dropna=False))
    print(f'Число найденных уникальных значений = {len(pd.unique(data[col]))}')

In [11]:
# Функция построения гистогаммы распределения признака col:

def  sign_hist(col):
    df = px.data.tips()
    fig = px.histogram(data, x=col)
    fig.show()

In [12]:
# Функция однократного кодирования признака col:

def  onehot_cod(col):
    encoder = ce.OneHotEncoder(cols=[col]) # указываем столбец для кодирования
    type_bin = encoder.fit_transform(data[col])
    data = pd.concat([data, type_bin], axis=1)
    data.drop([col], axis=1)

In [13]:
# Функция бинорного кодирования признака col:

def  binary_cod(col):
    bin_encoder = ce.BinaryEncoder(cols=[col]) # указываем столбец для кодирования
    type_bin = bin_encoder.fit_transform(data[col])
    data = pd.concat([data, type_bin], axis=1)
    data.drop([col], axis=1)

**1. status**

In [14]:
# объединим близкие по смыслу значений признаки по ключевым словам:

col = 'status'
lst = ['Active', 'Pending', 'Coming', 'Contract', 'Contingent', 'New', 'Backup', 'Pre-foreclosure', 'Sold', 'backups', 'Backups']
comb_val(col, lst)

In [15]:
# переназначим значения признака по словарю с учетом сокращений, регистра символов и присвоения пустым ячейкам значения Unknown:

col = 'status'
dict = {'Coming':'Coming soon', 'Apartment for rent':'For rent', 'P':'Pending', 'Ps':'Pending', 'pending':'Pending', 
        'Uc Continue To Show':'Contract', 'Under contract':'Contract', 'C Continue Show':'Contingent', 
        'Re Activated':'Reactivated', 'Pf':'Pending', 'Pi':'Pending', 'Ct':'Contract', 'Back On Market':'Back on Market',
        'For Sale':'For sale', 'for sale':'For sale', 'for rent':'For rent', 'C':'Contract', ' / auction':'Auction', 'nan':'Unknown', 
        'Condo for rent':'For rent', 'backups':'Backup', 'Backups':'Backup'}
remap_val(dict, col)

In [16]:
# выведем полученный перечень уникальных значений признака после преобразований:

col = 'status'
uniq_val_count(col)

For sale                            199520
Active                              106554
Unknown                              39256
Pending                               6889
New                                   6148
foreclosure                           5677
Pre-foreclosure                       3281
Contract                              3121
Auction                               1292
Contingent                             996
Price Change                           563
For rent                               412
Foreclosure                            343
Foreclosed                             294
Back on Market                         112
Coming soon                            110
Listing Extended                        28
Due Diligence Period                    27
CT Insp - Inspection Contingency        10
Contingency 48 Hr (+/ )                  5
Closed                                   5
Accepted Offer                           4
Backup                                   4
Reactivated

In [17]:
# # Построим гистогамму распределения признака col:

# col = 'status'
# sign_hist(col)

In [18]:
# # используем двоичное кодирование для кодировки признака status

# on_cod(col)
# data.head()

**2. private pool, PrivatePool**

In [19]:
col = 'private pool'
uniq_val_count(col)

NaN    370504
Yes      4151
Name: private pool, dtype: int64
Число найденных уникальных значений = 2


In [20]:
# Проверим есть ли пересечения непустых значений в столбцах private pool, PrivatePool

mask1 = data['private pool'].notna()
mask2 = data['PrivatePool'].notna()
data[mask1 & mask2].shape[0]

0

In [21]:
# т.к. пересечений непустых значений нет объединим данные о наличии бассейна в один столбец

data['PrivatePool'] = data['PrivatePool'].fillna(data['private pool'])
data = data.drop(['private pool'], axis=1)
data['PrivatePool'] = data['PrivatePool'].replace({'yes': 'Yes'})
data['PrivatePool'] = data['PrivatePool'].fillna(value='No')

In [22]:
# выведем полученный перечень уникальных значений признака после преобразований:

col = 'PrivatePool'
uniq_val_count(col)

No     330384
Yes     44271
Name: PrivatePool, dtype: int64
Число найденных уникальных значений = 2


In [23]:
# # Построим гистогамму распределения признака col:

# col = 'PrivatePool'
# sign_hist(col)

In [24]:
# # используем однократное кодирование для кодировки признака PrivatePool

# onehot_cod(col)
# data.head()

**3. propertyType**

In [25]:
col = 'propertyType'
uniq_val_count(col)

single-family home                                             91370
Single Family                                                  61886
NaN                                                            34554
Single Family Home                                             31725
condo                                                          25874
                                                               ...  
1 Story, Contemporary, Other (See Remarks)                         1
Custom, Elevated, Other                                            1
Contemporary, Farmhouse                                            1
2 Stories, Traditional, Mediterranean, Texas Hill Country          1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1280, dtype: int64
Число найденных уникальных значений = 1280


**4. street**

In [26]:
# выведем количество уникальных значений признака street:
col = 'street'
print(f'Число найденных уникальных значений = {len(pd.unique(data[col]))}')

Число найденных уникальных значений = 334752


In [27]:
# # выведем полученный перечень уникальных значений признака street:

# uniq_val_count(col)

In [28]:
# data.drop(['street'], axis=1)

In [29]:
data = data.drop(['street'], axis=1)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        374655 non-null  object
 1   propertyType  340101 non-null  object
 2   baths         269308 non-null  object
 3   homeFacts     374655 non-null  object
 4   fireplace     102519 non-null  object
 5   city          374621 non-null  object
 6   schools       374655 non-null  object
 7   sqft          334560 non-null  object
 8   zipcode       374655 non-null  object
 9   beds          283726 non-null  object
 10  state         374655 non-null  object
 11  stories       224902 non-null  object
 12  mls-id        24937 non-null   object
 13  PrivatePool   374655 non-null  object
 14  MlsId         310187 non-null  object
 15  target        374655 non-null  object
dtypes: object(16)
memory usage: 48.6+ MB
